()

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

data = pd.read_csv('https://raw.githubusercontent.com/pavlicag/campaign_finance_donation_optimization/main/538_FEC_Won_Opponent_Combined_Dataset.csv')
data

ModuleNotFoundError: No module named 'sklearn'

# Find best features

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
import numpy as np

subdata = data.drop(['Won?', 'Unnamed: 0', 'branch', 'district', 'forecastdate', 'name', 'timestamp', 'party'], axis=1)

count = 0
for i in subdata['Cand_Incumbent_Challenger_Open_Seat']:
    if i == 'INCUMBENT':
        subdata['Cand_Incumbent_Challenger_Open_Seat'][count] = 1
    else:
        subdata['Cand_Incumbent_Challenger_Open_Seat'][count] = 0
    count += 1

# Extract the features and target from your 'data' dataset
X = subdata
y = binary[0].ravel() # 1 if the candidate won, 0 otherwise

# create logistic regression object
logreg = LogisticRegression()

# create RFECV object
rfecv = RFECV(estimator=logreg, step=1, cv=5, scoring='accuracy')

# fit RFECV on data
rfecv.fit(X, y)

# print the best combination of variables
print("Optimal number of features : %d" % rfecv.n_features_)
print("Best features : ", np.where(rfecv.support_ == True)[0])


# Incumbent only model

In [ ]:
binary = []
for i in data['Won?']:
    if i == True:
        binary.append(1)
    else:
        binary.append(0)
binary = pd.DataFrame(binary)

binary_in = []
for i in data['Cand_Incumbent_Challenger_Open_Seat']:
    if i == 'INCUMBENT':
        binary_in.append(1)
    else:
        binary_in.append(0)
binary_in = pd.DataFrame(binary_in)

# Load your data into X (features) and y (labels)
X = binary_in # total money fundraised
y = binary[0].ravel() # 1 if the candidate won, 0 otherwise

# Calculate the average accuracy over 1000 runs
accs = []
for i in range(1000):
    # Split your data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = X_train.values.reshape(-1, 1)

    # Create a logistic regression model and train it on your data
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions on your test set
    y_pred = model.predict(X_test.values.reshape(-1, 1))

    # Evaluate your model's accuracy and append to the list of accuracies
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)

# Calculate the average accuracy over 100 runs
avg_acc = np.mean(accs)
print("Average Accuracy:", avg_acc)


# % Cash Advantage Only Model

In [ ]:
binary = []
for i in data['Won?']:
    if i == True:
        binary.append(1)
    else:
        binary.append(0)
binary = pd.DataFrame(binary)

# Load your data into X (features) and y (labels)
X = data['Percent_Cash_Advantage'] # total money fundraised
y = binary[0].ravel() # 1 if the candidate won, 0 otherwise

# Calculate the average accuracy over 1000 runs
accs = []
for i in range(1000):
    # Split your data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = X_train.values.reshape(-1, 1)

    # Create a logistic regression model and train it on your data
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions on your test set
    y_pred = model.predict(X_test.values.reshape(-1, 1))

    # Evaluate your model's accuracy and append to the list of accuracies
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)

# Calculate the average accuracy over 100 runs
avg_acc = np.mean(accs)
print("Average Accuracy:", avg_acc)


# Best EV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Extract the best features from your dataset
X = data[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]
y = binary[0].ravel() # 1 if the candidate won, 0 otherwise

# Calculate the average accuracy over 1000 runs
accs = []
for i in range(1000):
    # Split your data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Create a logistic regression model and train it on your data
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions on your test set
    y_pred = model.predict(X_test)

    # Evaluate your model's accuracy and append to the list of accuracies
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)

# Calculate the average accuracy over 100 runs
avg_acc = np.mean(accs)
print("Average Accuracy:", avg_acc)


# Using best EV intial prediction percentage

In [ ]:
# Load your data into X (features) and y (labels)
X = data[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage', 'name']] # total money fundraised
y = data[['Won?', 'name']] # 1 if the candidate won, 0 otherwise

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Extract the 'name' column from the test set for output purposes
names = X_test['name']

# Remove the 'name' column from the X_train and X_test sets
X_train = X_train[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]
X_test = X_test[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]

# Create a logistic regression model and train it on your data
model = LogisticRegression()
model.fit(X_train, y_train['Won?'])

# Make predictions on your test set
y_pred_proba = model.predict_proba(X_test)

# Output the predicted probabilities alongside the candidate names
output_int = pd.concat([names.reset_index(drop=True), pd.DataFrame(y_pred_proba)], axis=1)
output_int.columns = ['name', 'probability_of_losing', 'probability_of_winning']
output_int

# Using best EV final prediction percentage

In [ ]:
# Load your data into X (features) and y (labels)
X = data[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage', 'name']] # total money fundraised
y = data[['Won?', 'name']] # 1 if the candidate won, 0 otherwise

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Extract the 'name' column from the test set for output purposes
names = X_test['name']

# Remove the 'name' column from the X_train and X_test sets
X_train = X_train[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]
X_test = X_test[['Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]

X_train += 2900
X_test += 2900

# Create a logistic regression model and train it on your data
model = LogisticRegression()
model.fit(X_train, y_train['Won?'])

# Make predictions on your test set
y_pred_proba = model.predict_proba(X_test)

# Output the predicted probabilities alongside the candidate names
output_fin = pd.concat([names.reset_index(drop=True), pd.DataFrame(y_pred_proba)], axis=1)
output_fin.columns = ['name', 'probability_of_losing', 'probability_of_winning']
output_fin

# Difference in probabilites over test dataset

In [ ]:
# Merge the two dataframes on the 'name' column
merged_df = output_fin.merge(output_int, on='name', suffixes=('_fin', '_int'))

# Calculate the difference between the two probabilities
merged_df['probability_difference'] = merged_df['probability_of_winning_fin'] - merged_df['probability_of_winning_int']

# Select the columns you want to keep in the final dataframe
final_df = merged_df[['name', 'probability_difference']]

# Print the final dataframe
final_df


# EV for entire dataset

In [ ]:
# Load your data into X (features) and y (labels)
X = data[['win_pct', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage', 'name']] # total money fundraised
y = data[['Won?', 'name']] # 1 if the candidate won, 0 otherwise

# Remove the 'name' column from the X set
X = X[['win_pct', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]

# Create a logistic regression model and train it on your data
model = LogisticRegression()
model.fit(X, y['Won?'])

# Make predictions on the entire dataset
y_pred_proba = model.predict_proba(X)

# Output the predicted probabilities alongside the candidate names
output_int = pd.concat([y['name'].reset_index(drop=True), pd.DataFrame(y_pred_proba)], axis=1)
output_int.columns = ['name', 'probability_of_losing', 'probability_of_winning']
output_int

# EV for entire dataset + donation

In [ ]:
# Load your data into X (features) and y (labels)
X = data[['win_pct', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage', 'name']] # total money fundraised
y = data[['Won?', 'name']] # 1 if the candidate won, 0 otherwise

# Remove the 'name' column from the X set
X = X[['win_pct', 'Total_Disbursement', 'Cash_On_Hand_COP', 'Cash_Advantage']]
X += 3300
# Create a logistic regression model and train it on your data
model = LogisticRegression()
model.fit(X, y['Won?'])

# Make predictions on the entire dataset
y_pred_proba = model.predict_proba(X)

# Output the predicted probabilities alongside the candidate names
output_fin = pd.concat([y['name'].reset_index(drop=True), pd.DataFrame(y_pred_proba)], axis=1)
output_fin.columns = ['name', 'probability_of_losing', 'probability_of_winning']
output_fin

# Difference in prob

In [ ]:
# Merge the two dataframes on the 'name' column
merged_df = output_fin.merge(output_int, on='name', suffixes=('_fin', '_int'))

# Calculate the difference between the two probabilities
merged_df['probability_difference'] = merged_df['probability_of_winning_fin'] - merged_df['probability_of_winning_int']

# Select the columns you want to keep in the final dataframe
final_df = merged_df[['name', 'probability_difference']]

# Print the final dataframe
final_df


In [ ]:
output_diff_sorted = final_df.sort_values(by='probability_difference')


data["probability_difference"] = final_df["probability_difference"]
sorted_data = data.sort_values(by='probability_difference',ascending = False).reset_index(drop=True)
display(sorted_data.head(30))
sorted_data.to_csv("regression_data.csv")


In [ ]:
binary = []
for i in data['Won?']:
    if i == True:
        binary.append(1)
    else:
        binary.append(0)
binary = pd.DataFrame(binary)

# Load your data into X (features) and y (labels)
X = data['win_pct'] # total money fundraised
y = binary[0].ravel() # 1 if the candidate won, 0 otherwise

# Calculate the average accuracy over 1000 runs
accs = []
for i in range(1000):
    # Split your data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = X_train.values.reshape(-1, 1)

    # Create a logistic regression model and train it on your data
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions on your test set
    y_pred = model.predict(X_test.values.reshape(-1, 1))

    # Evaluate your model's accuracy and append to the list of accuracies
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)

# Calculate the average accuracy over 100 runs
avg_acc = np.mean(accs)
print("Average Accuracy:", avg_acc)
